In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('/content/gdrive/My Drive/InformationRetrieval/Assignment_1')

In [ ]:
!pip install num2Words
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
from num2words import num2words
from nltk.stem import WordNetLemmatizer
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

     |████████████████████████████████| 102kB 3.7MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


#Getting Folders

In [ ]:
title = "stories"
alpha = 0.3
folders = [x[0] for x in os.walk(str(os.getcwd())+'/'+title+'/')]
folders[0] = folders[0][:len(folders[0])-1]
print(folders)

['/content/gdrive/My Drive/InformationRetrieval/Assignment_1/stories', '/content/gdrive/My Drive/InformationRetrieval/Assignment_1/stories/FARNON', '/content/gdrive/My Drive/InformationRetrieval/Assignment_1/stories/SRE']


#Preparing Dataset 

In [ ]:
dataset = []
c = False
for i in folders:
    file = open(i + "/index.html", 'r')
    text = file.read().strip()
    file.close()

    file_name = re.findall('><A HREF="(.*)">', text)
    file_title = re.findall('<BR><TD> (.*)\n', text)

    if c == False:
        file_name = file_name[2:]
        c = True

    # print(len(file_name), len(file_title))

    for j in range(len(file_name)):
        dataset.append((str(i) + "/" + str(file_name[j]), file_title[j]))

# print(dataset)
N = len (dataset)
print(N)

467


In [ ]:
# importing libraries
import numpy as np
import os
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from natsort import natsorted
import string
lemmatizer = WordNetLemmatizer()

In [ ]:
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)

# Convert the text to lower case
def convert_lower_case(data):
    return np.char.lower(data)

# Remove stopwords from tokens
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
#     print(np.char.strip(new_text))
    return np.char.strip(new_text)

# Remove punctuation marks from tokens
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

def preprocess(data, query):
    data = convert_lower_case(data)
#     print("data2 =",data)
    data = remove_punctuation(data) #remove comma seperately
#     print("data3 =",data)
    data = remove_apostrophe(data)
#     print("data4 =",data)
    data = remove_stop_words(data)
#     print("data5 =",data)
    data = convert_numbers(data)
    return data

In [ ]:
test_before = "DING DONG! The melancholy tintinabulation"

In [ ]:
print('Before Preprocessing : {}'.format(test_before))
print('After Preprocessing : {}'.format(preprocess(test_before, False)))

Before Preprocessing : DING DONG! The melancholy tintinabulation
After Preprocessing :  ding dong melancholy tintinabulation


In [ ]:
# Initialize the file no.
fileno = 0  
# Initialize the dictionary.
pos_index = {}  
# Initialize the file mapping (fileno -> file name).
file_map = {}
doc = 0
for i in dataset:
    file = open(i[0], 'r', encoding='cp1250', errors='ignore')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)

    if doc % 100 == 0:
        print(doc)

    tokens = word_tokenize(str(preprocessed_text))


    for pos, term in enumerate(tokens):
      
      term = lemmatizer.lemmatize(term)
      
      if term in pos_index:
        pos_index[term][0] = pos_index[term][0]+1

        if(fileno in pos_index[term][1]):
          pos_index[term][1][fileno].append(pos)
        else:
          pos_index[term][1][fileno] = [pos]
        
      else:
        pos_index[term] = []
        pos_index[term].append(1)
        pos_index[term].append({})
        pos_index[term][1][fileno] = [pos]
    
    file_map[fileno] = i[1]
    fileno = fileno+1

    doc = doc+1

0
100
200
300
400


In [ ]:
# file_name = "positional_index.pkl"

# open_file = open(file_name, "wb")
# pickle.dump(pos_index, open_file)
# open_file.close()

In [ ]:
file_name = "positional_index.pkl"
open_file = open(file_name, "rb")
pos_index = pickle.load(open_file)
open_file.close()

In [ ]:
print(pos_index['good'])

[2409, {0: [234, 676], 1: [30, 40, 85, 132, 238, 305], 6: [853], 7: [800, 895], 8: [338, 739, 741, 750, 1009, 1686, 1732, 1948, 2113, 2308, 2435, 2528, 2826], 9: [517], 11: [555, 915, 1232, 1624, 1828, 1835, 1884, 1901, 2032, 2835], 13: [81, 110, 337, 696], 14: [990, 2519], 16: [1433, 1468, 1558, 1560, 1664, 1990, 2563, 3416, 3541, 3903, 3961], 17: [202], 18: [72, 174, 387, 428, 484, 495, 563, 851, 1347, 1389, 1408, 1439, 1460, 1535, 1808, 2167, 2809, 3460, 3464, 3497], 19: [548, 829], 20: [1442, 1578, 1860, 2075, 3296, 3469, 3631, 3666, 3928], 22: [1091, 1146], 23: [612], 26: [42, 135, 1336, 1444, 1905, 2112, 2563, 3078, 3341, 3386, 3877, 4447, 4772, 4829, 4851, 5803, 6097, 6559, 6937, 7200, 7893, 8247, 8255, 8612, 9828, 10237, 10550, 10557, 10566, 11085, 11645, 11662, 11676, 11695, 12120, 12245, 12345, 12556, 12558, 12571, 12607, 12623, 12648, 12746, 14068, 14696, 14825, 15517], 27: [781, 819, 1569, 2123, 2226, 3102, 3281, 3358, 4055, 4057, 4077, 4085, 4445, 4469, 4755, 5701], 31: [7

In [ ]:
test_phrase = "100 west by 53 north"
preprocessed_text = preprocess(test_phrase, False)
print(preprocessed_text)
test_phrase_lemmat = [lemmatizer.lemmatize(each_word) for each_word in word_tokenize(str(preprocessed_text))]
# test_phrase_lemmat =  word_tokenize(str(preprocessed_text))
print(test_phrase_lemmat)
result = {}
count = 0

 one hundred west fifty three north
['one', 'hundred', 'west', 'fifty', 'three', 'north']


In [ ]:
for item in test_phrase_lemmat :
    
    my_dictionary = pos_index[item][1]
    
    if len(result) == 0 :
        result = my_dictionary
    else:
        document_list = list(set(result.keys()) & set(my_dictionary.keys()))
        temp = {}
        
        for doc in document_list :
            temp[doc] = result[doc]    
        result = {}    
        result = temp
        if len(document_list) == 0:
            print("No documents")
            break
        
        for doc in document_list :
            
            list_posting = result[doc]
            temp_pos = []
            
            for pos in list_posting:
                
                if pos+count in my_dictionary[doc]:
                    temp_pos.append(pos)
            
            result[doc] = temp_pos
            
            if len(result[doc]) == 0 :
                del result[doc]
        
    count = count + 1

In [ ]:
# Length of documents retrived
print("Number of Documents retrived are :" ,len(result))

Number of Documents retrived are : 1


In [ ]:
for id in list(result.keys()):
  print(dataset[id][0].split('/')[6:])

['stories', '100west.txt']


In [ ]:
result

{15: [33]}

In [ ]:
print_doc(15)

('/content/gdrive/My Drive/InformationRetrieval/Assignment_1/stories/6ablemen.txt', 'The Story of the Six Able Men')
SIX ABLE MEN  

   Once upon a time there lived a young soldier named Martin who had enlisted 
in the royal army to flght a war. The war was long but victorious and when the
King abandoned the enemy's territory and returned with his troops to the 
homeland, he left Martin to guard the only bridge on the river that separated 
the two nations.
   "Stay on watch on the bridge," the King ordered. "Don't let any enemy 
soldier go by." Days and then months passed, and the soldier kept his watch on
the bridge. He survived by asking the passers-by for food and, after two 
years, thought that the authorities had probably forgotten him. He then headed
towards the capital, where he would ask the King for all his back pay. His 
pockets were empty and his only possessions were a pipe, a bit of tobacco and 
his sword.
   A couple of days later he arrived in a valley where a stream cro